#Installed pymysql to make connection with mysql

In [ ]:
#pip install pymysql

In [1]:
#Imported libraries:
import pymysql
import pandas as pd

# Connection with mysql

In [2]:
#Making connection with the client database using authentications:
conn = pymysql.connect(host=,  
                       port=,  
                       user=,  
                       passwd=,  
                       db=,  
                       charset='utf8')  

In [3]:
#Queries to fetch data from database:
sql_con = "select * from consumer"
sql_con_prod = "select * from consumer_product"
sql_con_ser = "select * from consumer_servicerequest"
sql_sold = "select * from sold_plan"

In [4]:
#Reading SQl into Dataframe:
df_con = pd.read_sql(sql_con,conn)
df_con_prod = pd.read_sql(sql_con_prod,conn)
df_con_serv = pd.read_sql(sql_con_ser,conn)
df_sold = pd.read_sql(sql_sold,conn)

In [ ]:
#Writing Dataframe to csv, So that it can be used with Tableau:
df_con.to_csv(r'C:\Users\Mangesh\Desktop\df_con.csv', index=True, header=True)
df_con_prod.to_csv(r'C:\Users\Mangesh\Desktop\df_con_prod.csv', index=True, header=True)
df_con_serv.to_csv(r'C:\Users\Mangesh\Desktop\df_con_serv.csv', index=True, header=True)
df_sold.to_csv(r'C:\Users\Mangesh\Desktop\df_sold.csv', index=True, header=True)

# The percentage of service requests raised under a plan of the total number of requests raised.

Requirement says:
    1. Count the total no of Consumer Service Request raised.
    2. Count the no. of Consumer Service Request under individual planID.
    3. Then Count the percentage of 'ConsumerServiceRequest raised under individual PlanID' of 
    'Total no of ConsumerServiceRequest raised'  
-To fulfill above requirement we need only three columns 'ConsumerRequestID', 'SoldPlanID' and 'PlanID' from two tables 
'consumer_servicerequest' and 'sold_plan'

-'SoldPlanID' is the foreign key.

-I have used 'Right Outer Join' as 'PlanID' is important beacause we need to seggregate on the basis of 'PlanID'.

-And in table 'consumer_servicerequest', column 'PlanID' is not mapped with column 'ConsumerServiceRequestID'.

-I found many '0' values when I mapped thoese two tables, which suggests that there are many cells with 'ConsumerServiceRequestID' but no 'PlanID' mapped to it. So I dropped 'NA' values.

- Then I simply take shape[0] id of the Dataframe, which gave me 'total no of service request raised'.

- Then I groupby='PlanID' so I got 'no of records of service request raised' under each 'PlanID'.

- Then I performed simple percentage operation on two values.

In [5]:
# Query for fetching data, related to ConsumerRequest, SoldPlan and PLanID:

sql_per_serv = "select cs.ConsumerServiceRequestID, sp.SoldPlanID, sp.PlanID from consumer_servicerequest as cs right outer join sold_plan as sp on cs.SoldPlanID = sp.SoldPlanID;"


In [6]:
#read data
df_per_serv = pd.read_sql(sql_per_serv,conn)

In [7]:
#Drop NA values
df_per_serv = df_per_serv.dropna(how='any', axis=0)

In [8]:
#Total No of service request (i.e Row count) 
total_req = df_per_serv.shape[0]

In [9]:
#GroupBy on the basis of 'PlanID'
df_per_serv = df_per_serv.groupby('PlanID', as_index=False).count()

In [10]:
#Percentage Operation
print("The percentage of service requests raised under a plan of the total number of requests raised: \n")
print ("Total No of requests raised:{} \n".format(total_req))
for i in range(df_per_serv.shape[0]):    
    per = round(((df_per_serv.ConsumerServiceRequestID[i]*100)/total_req),2)
    print("PlnaID {} : Percentage of requests raised {} %".format(df_per_serv['PlanID'][i],per))

The percentage of service requests raised under a plan of the total number of requests raised: 

Total No of requests raised:9305 

PlnaID 1 : Percentage of requests raised 5.44 %
PlnaID 2 : Percentage of requests raised 5.35 %
PlnaID 3 : Percentage of requests raised 5.26 %
PlnaID 4 : Percentage of requests raised 5.22 %
PlnaID 5 : Percentage of requests raised 5.09 %
PlnaID 6 : Percentage of requests raised 5.21 %
PlnaID 7 : Percentage of requests raised 5.15 %
PlnaID 8 : Percentage of requests raised 5.35 %
PlnaID 9 : Percentage of requests raised 5.17 %
PlnaID 10 : Percentage of requests raised 5.27 %
PlnaID 11 : Percentage of requests raised 5.17 %
PlnaID 12 : Percentage of requests raised 5.35 %
PlnaID 13 : Percentage of requests raised 4.95 %
PlnaID 14 : Percentage of requests raised 5.43 %
PlnaID 15 : Percentage of requests raised 5.27 %
PlnaID 16 : Percentage of requests raised 5.39 %
PlnaID 17 : Percentage of requests raised 5.29 %
PlnaID 18 : Percentage of requests raised 5.

# The average number of plans sold per week

Requirement says:
    1. Count the total no of plans sold.
    2. Count the number of weeks in total dataset from 'DateOfPurchase'
    
- I need only one table 'sold_plan' and two columns 'SoldPlanID' and 'DateOfPurchase'.

- I have added two columns in to the dataset 1.'Year' & 2. 'Week'.

- Groupby the Data on two columns - 'year' and 'week' - In each yaer there are many weeks.

- length of the Dataframe gave me 'Total no of weeks'.

- shape[0] (I.e NO of rows) of initial Dataframe gave me 'Total No of plans sold'

- Then simple average calculation. 


In [11]:
#Query to find the reliable data with 'SoldPlan' and 'DateOfPurchase'
sql_avg_plan = "select sp.SoldPlanID, sp.DateOfPurchase from sold_plan as sp"

In [12]:
#Dataframe 
def_avg_plan = pd.read_sql(sql_avg_plan,conn)

In [13]:
#Adding column with number of week in the year from 'DateOfPurchase'
def_avg_plan['week']=def_avg_plan.DateOfPurchase.dt.weekofyear

In [14]:
#Adding another column with year from 'DateOfPurchase'
def_avg_plan['year']=def_avg_plan.DateOfPurchase.dt.year

In [15]:
# Groupby the Data on two columns - 'year' and 'week' - In each yaer there are many weeks.
# Taking length of the Dataframe gives - total no of weeks present in the dataset.
no_of_week = len(def_avg_plan.groupby(['year','week']))


In [16]:
# Calculating Average:
avg = def_avg_plan.shape[0]/no_of_week
print("The average number of plans sold per week :~", round(avg))

The average number of plans sold per week :~ 189


# The brand (BrandID) with the highest number of plans bought by customers

Requirement says:
    1. Highest no of plans sold.
    2. Brand with highest no of plans sold.
- I have used two tables -  1.consumer_product
                            2.sold_plan

-and two columns 1.BRandID 
                 2. PlanID
    
- I have used simple inner join as we need only thoese 'BRandID' which has 'PlanID'.

- Then Groupby the dataset on the basis of 'BrandID'. 

- Count() - Method gave me no of PLanID under each BrandID.

- I checked for the max() 'PLanID' number (i.e sum) And BrandID attached to it, is our final answer. 

In [17]:
# query
sql_brand  = "select cp.BrandID, sp.PlanID from  consumer_product as cp inner join sold_plan as sp on cp.ConsumerID = sp.ConsumerID AND cp.ConsumerProductID = sp.ConsumerProductID;"


In [18]:

df_brand = pd.read_sql(sql_brand,conn)


In [19]:
#Groupby
df_brand = df_brand.groupby('BrandID', as_index=False).count()

In [20]:
#Max() of PlanID total numbers under each Brand:
print("The brand (BrandID) with the highest number of plans bought by customers : ",df_brand[df_brand['PlanID'] == df_brand['PlanID'].max()]['BrandID'].to_string(index=False))

The brand (BrandID) with the highest number of plans bought by customers :   16


# Connection close

In [22]:
conn.close()